In [1]:
from dotenv import load_dotenv
from core import init_requests_cache

# config
init_requests_cache()
load_dotenv()


True

In [2]:
from core import CompanyProduct

target = CompanyProduct.same("98point6")

In [3]:
from crunchbase import find_people_url

url = find_people_url(target)

In [4]:
import scrapfly_scrapers.crunchbase
scrapfly_scrapers.crunchbase.BASE_CONFIG["cache"] = True

crunchbase_raw_response = await scrapfly_scrapers.crunchbase.scrape_company(url)

# pprint(crunchbase_response)

2024-08-13 11:03:43.715 | INFO     | scrapfly_scrapers.crunchbase:scrape_company:60 - scraping company: https://www.crunchbase.com/organization/98point6/people


In [5]:
from pprint import pprint
pprint(crunchbase_raw_response)

{'employees': [{'job_departments': ['legal', 'management'],
                'job_levels': ['l_100_individual', 'l_500_exec'],
                'linkedin': 'aimee-brice-0588a19',
                'name': 'Aimee Brice'},
               {'job_departments': ['finance'],
                'job_levels': ['l_100_individual', 'l_500_exec'],
                'linkedin': 'angelo-gray-4483404',
                'name': 'Angelo Gray'},
               {'job_departments': ['management'],
                'job_levels': ['l_500_exec'],
                'linkedin': 'anifatbalogun',
                'name': 'Anifat Balogun'},
               {'job_departments': ['management', 'medical_science'],
                'job_levels': ['l_500_exec'],
                'linkedin': 'younggren',
                'name': 'Bradley Younggren'},
               {'job_departments': None,
                'job_levels': ['l_500_exec'],
                'linkedin': 'bradley-younggren-b0244312',
                'name': 'Bradley Younggren'},

In [6]:
from crunchbase.models import parse

organization, employees = parse(crunchbase_raw_response)

In [10]:
import jinja2
from datetime import datetime

env = jinja2.Environment(loader=jinja2.FileSystemLoader("templates"))

print(env.get_template("crunchbase.md").render(organization=organization, employees=employees, current_year=datetime.now().year))

# 98point6 Inc., founded 2015-03-01 ((Crunchbase, 2024)[https://www.crunchbase.com/organization/98point6])
98point6 is a digital care service that delivers personalized consultation, diagnosis, and treatment to patients. The company combines deep technology with the expertise of board-certified physicians to provide text-based care anytime, anywhere. The company meets consumers where they are by offering private, text-based diagnosis and treatment via a mobile app. 

The company was founded in 2015 and is headquartered in Seattle, Washington.

Website: http://www.98point6.com
LinkedIn: https://www.linkedin.com/company/98point6-inc.
Twitter: https://twitter.com/98point6inc
Facebook: https://www.facebook.com/98point6inc

## Funding (299M USD total)

- 32M USD on 2023-04-12
- 20M USD on 2022-09-01
- 118M USD on 2020-10-15
- 43M USD on 2020-04-03
- 50M USD on 2018-10-30
- 20M USD on 2017-09-19
- 12M USD on 2016-07-18
- 3M USD on 2015-04-17


## News

- 98point6 lays off employees after shi

In [8]:
organization.id, url

('98point6', 'https://www.crunchbase.com/organization/98point6/people')